In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [2]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold
import json

/home/dmitriishubin/anaconda3/envs/thesis/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
img = nib.load('../data/Task02_Heart/imagesTr/la_003.nii.gz')
img = np.array(img.dataobj)

In [4]:
img.shape

(320, 320, 130)

In [13]:
patients = [i for i in os.listdir('../data/Task02_Heart/imagesTr/') if i.find('._la')==-1]

In [14]:
patients

['la_026.nii.gz',
 'la_005.nii.gz',
 'la_007.nii.gz',
 'la_019.nii.gz',
 'la_009.nii.gz',
 'la_016.nii.gz',
 'la_004.nii.gz',
 'la_022.nii.gz',
 'la_018.nii.gz',
 'la_014.nii.gz',
 'la_011.nii.gz',
 'la_010.nii.gz',
 'la_024.nii.gz',
 'la_020.nii.gz',
 'la_023.nii.gz',
 'la_029.nii.gz',
 'la_030.nii.gz',
 'la_003.nii.gz',
 'la_021.nii.gz',
 'la_017.nii.gz']

In [7]:
patients[0][:-7]

'la_026'

In [28]:
names = []

os.makedirs('../data/Task02_Heart/processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'../data/Task02_Heart/imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'../data/Task02_Heart/labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'../data/Task02_Heart/processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i].reshape(320, 320,1)
        temp_seg = seg[:,:,i].reshape(320, 320,1)
        
        
        np.save(f'../data/Task02_Heart/processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'../data/Task02_Heart/processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [15]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [36]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task02_Heart/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task02_Heart/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
    
    
    images_val = []
    for patient in patients_test:
        images_val+=[
                f'../data/Task02_Heart/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task02_Heart/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
        
    
    split = {
        'train': images_train,
        'val': images_val,
    }
    
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [19]:
f'../data/Task02_Heart/processed_data/{patients_train}/'

"../data/Task02_Heart/processed_data/['la_007' 'la_019' 'la_009' 'la_016' 'la_004' 'la_022' 'la_018' 'la_014'\n 'la_011' 'la_010' 'la_024' 'la_020' 'la_023' 'la_030' 'la_021' 'la_017']/"